In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import semopy

from BSA import ExploratoryAnalysis

In [3]:
# to get the data

expl = ExploratoryAnalysis()
expl.ExploratoryReport('data.xlsx')

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [4]:
data = expl.clean_data
#data

# SEMOPY

In [21]:
sem_model_description = '''
Power ~ Meaningful + Different + Salient
'''
sem_model = semopy.Model(sem_model_description)

sem_model.fit(data)

SolverResult(fun=2.329693016278611e-08, success=True, n_it=21, x=array([ 9.830368  ,  4.30392963,  7.65398318, 30.07126419]), message='Optimization terminated successfully', name_method='SLSQP', name_obj='MLW')

In [18]:
sem_model.inspect()

,lval,op,rval,Estimate,Std. Err,z-value,p-value
0,Power,~,Meaningful,9.830368,0.044152,222.648922,0.0
1,Power,~,Different,4.303930,0.044152,97.480104,0.0
2,Power,~,Salient,7.653983,0.044152,173.355779,0.0
3,Power,~~,Power,30.071264,0.342405,87.823687,0.0


# PGMPY SEM

In [70]:
# не заработало

In [41]:
import pgmpy
from pgmpy.models import SEM, SEMGraph
from pgmpy.estimators import SEMEstimator, IVEstimator

pgmpy.config.set_backend('torch')


In [69]:
data[vars_].mean()

Different     2.009595e-12
Meaningful   -1.231688e-12
Power         1.296512e+01
Salient       2.463371e-12
dtype: float64

In [ ]:
ebunch_ = [
    ('Power', 'Meaningful'),
    ('Power', 'Different'), 
    ('Power', 'Salient'), 
]

vars_ = ['Different', 'Meaningful', 'Power', 'Salient']

#observed_ = ['Power', 'Meaningful', 'Different', 'Salient']
model = SEMGraph(ebunch_)
estimator = SEMEstimator(model).fit(data[vars_], method='ml')


In [106]:
pip list

Package                   Version
------------------------- ------------
anyio                     4.2.0
argon2-cffi               21.3.0
argon2-cffi-bindings      21.2.0
asttokens                 2.0.5
async-lru                 2.0.4
attrs                     23.1.0
Babel                     2.11.0
beautifulsoup4            4.12.2
bleach                    4.1.0
Brotli                    1.0.9
certifi                   2024.2.2
cffi                      1.16.0
charset-normalizer        2.0.4
colorama                  0.4.6
comm                      0.2.1
contourpy                 1.2.1
cycler                    0.12.1
daft                      0.1.2
debugpy                   1.6.7
decorator                 5.1.1
defusedxml                0.7.1
et-xmlfile                1.1.0
executing                 0.8.3
factor_analyzer           0.5.1
fastjsonschema            2.16.2
filelock                  3.13.3
fonttools                 4.50.0
fsspec                    2024.3.1
idna           

# PLSPM

In [5]:
import plspm.config as c
from plspm.plspm import Plspm
from plspm.scheme import Scheme
from plspm.mode import Mode

In [28]:
structure = c.Structure()

structure.add_path(['Different_', 'Meaningful_', 'Salient_'], ['Power_'])

for t in ['Different_', 'Meaningful_', 'Salient_']: 
    structure.add_path(['@input.Affinity_', '@input.MeetNeeds_', '@input.Dynamic_', '@input.Unique_'], [t])


config = c.Config(structure.path(), scaled=False)
config.add_lv("Power_", Mode.A, c.MV("Power"))
config.add_lv("Different_", Mode.A, c.MV("Different"))
config.add_lv("Meaningful_", Mode.A, c.MV("Meaningful"))
config.add_lv("Salient_", Mode.A, c.MV("Salient"))

config.add_lv("@input.Affinity_", Mode.A, c.MV("@input.Affinity"))
config.add_lv("@input.MeetNeeds_", Mode.A, c.MV("@input.MeetNeeds"))
config.add_lv("@input.Dynamic_", Mode.A, c.MV("@input.Dynamic"))
config.add_lv("@input.Unique_", Mode.A, c.MV("@input.Unique"))




plspm_calc = Plspm(data, config, Scheme.CENTROID)
#print(plspm_calc.inner_summary())
plspm_calc.path_coefficients()

ValueError: Latent variable Power_ is not listed in the outer model paths or higher order constructs.

In [26]:
structure = c.Structure()
structure.add_path(['Meaningful_', 'Salient_'], ['Power_'])
structure.add_path(['Different_'], ['Power_'])

for t in ['Different_', 'Meaningful_', 'Salient_']: 
    structure.add_path(['@input.Affinity_', '@input.MeetNeeds_', '@input.Dynamic_', '@input.Unique_'], [t])

config = c.Config(structure.path(), scaled=False)
config.add_lv("Power_", Mode.A, c.MV("Power"))
config.add_lv("Different_", Mode.A, c.MV("Different"))
config.add_lv("Meaningful_", Mode.A, c.MV("Meaningful"))
config.add_lv("Salient_", Mode.A, c.MV("Salient"))

config.add_lv("@input.Affinity_", Mode.A, c.MV("@input.Affinity"))
config.add_lv("@input.MeetNeeds_", Mode.A, c.MV("@input.MeetNeeds"))
config.add_lv("@input.Dynamic_", Mode.A, c.MV("@input.Dynamic"))
config.add_lv("@input.Unique_", Mode.A, c.MV("@input.Unique"))

#config.add_lv_with_columns_named("Power", Mode.A, data, "Power")
#config.add_lv_with_columns_named("Different", Mode.A, data, "Different")
#config.add_lv_with_columns_named("Meaningful", Mode.A, data, "Meaningful")
#config.add_lv_with_columns_named("Salient", Mode.A, data, "Salient")

plspm_calc = Plspm(data, config, Scheme.CENTROID)
#print(plspm_calc.inner_summary())
plspm_calc.path_coefficients()

,@input.Unique_,@input.Dynamic_,@input.MeetNeeds_,@input.Affinity_,Salient_,Meaningful_,Different_,Power_
@input.Unique_,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
@input.Dynamic_,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
@input.MeetNeeds_,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
@input.Affinity_,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
Salient_,-0.281309,-0.020722,0.234153,0.318354,0.000000,0.000000,0.000000,0.0
Meaningful_,0.519509,-0.423485,0.380830,0.456176,0.000000,0.000000,0.000000,0.0
Different_,-0.097617,1.260676,-0.158544,-0.335899,0.000000,0.000000,0.000000,0.0
Power_,0.000000,0.000000,0.000000,0.000000,0.536138,0.688587,0.301477,0.0
